## double alpha with priors functions

### specialized functions for double alpha with priors

In [1]:
def update_prob(recip, Prob, alpha):
    Prob = Prob + alpha*(recip-Prob)
    return Prob

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    prob = params[3:6]
    
    # pull EVs from priors
    ev = [update_value(prob[0]), update_value(prob[1]), update_value(prob[2])]
    
    totalLLH = 0 
    for trial in range(0, len(data)):
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        # use the probability of the selection (choice-probability) to update log likelihood
        if choice != 0:
            cprob = probs[choice-1] 
            #print(cprob, isinstance(cprob, float))

            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
        
            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], alpha)
            ev[trustee] = update_value(prob[trustee])
        
    return totalLLH

def model_fit(data):
    
    tries = 100 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        a_gain=random.uniform(0, 1)
        a_loss=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        prob1=random.uniform(0,1)
        prob2=random.uniform(0,1)
        prob3=random.uniform(0,1)
        params = [a_gain, a_loss, beta, prob1, prob2, prob3]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 20), (0, 1), (0, 1), (0, 1)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
        if bestFit == 'NA':
            line = ['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']
        else:
            line = np.append(bestFit['x'], bestFit['fun'])
            
    return bestFit